In [33]:
from boxsdk import OAuth2, Client
from pathlib import Path
import time
import multiprocessing

auth = OAuth2(
    client_id='mppopj23ems9esarsoz0uymdtbbmnn2m',
    client_secret='J4oYDvND78wQWsmcEJxjvyz5CsKvH7Iz',
    access_token='6TtykhjnjdUWpSzRXKpeJpfsRBUQAgAI',
)

client = Client(auth)

user = client.user().get()
print('The current user ID is {0}'.format(user.id))
print(user)

The current user ID is 6936353235
<Box User - 6936353235 (Joseph Gaustad)>


In [2]:
## using JWTAUth
from boxsdk import JWTAuth, Client
config = JWTAuth.from_settings_file('81479_7ydilrbs_config.json')
client = Client(config)
user = client.user().get()
print('The current user ID is {0}'.format(user.id))
print(user)

FileNotFoundError: [Errno 2] No such file or directory: '81479_7ydilrbs_config.json'

In [29]:
def downloader(path, item):
    #print('\r', end = '')
    print(path/item.name, 'starting download.')

    with open(path/str(item.name), 'wb') as f:
        item.download_to(f)
        #print('Downloaded', target_folder/str(item.name))
        
    #print('\r', end = '')

In [35]:
#toy example
items = ['655895782085', '655896664668', '655895351039', '655896434822', '655895565256']
path = Path('/Volumes/2TB_nvme/init_box_download_multi')

def download_file(fileid):
    item = client.file(file_id=fileid).get()
    with open(path/str(item.name), 'wb') as f:
        item.download_to(f)

download_file('655895782085')

In [37]:
#multiprocessing
if __name__ == '__main__':
    starttime = time.time()
    pool = multiprocessing.Pool(5)
    pool.map(download_file, items)
    pool.close()
    print(f'{len(items)} items downloaded in {path} in {(time.time() - starttime)} seconds.')




5 items downloaded in /Volumes/2TB_nvme/init_box_download_multi in 201.5202260017395 seconds.


In [31]:
def copy_box(target_folder, box_folder = '93670449030'):
    '''This function takes a string as a target directory to download a the box folder contents.  default ID is the `Environmental_Data_MASTER`'''
    # use pathlib to create path and directory if necessary.
    path = Path(target_folder)
    if not path.is_dir():
        path.mkdir()
        
    # get all items in box folder
    folder_items = client.folder(folder_id=box_folder).get_items()
    
    # iterate through each item
    files = []
    sub_dirs = []
    for item in folder_items:
        if item.type == 'folder': sub_dirs.append(item)
        
        elif Path(path/str(item.name)).is_file():
            continue
        
        elif item.type == 'file': files.append((path,item))
            
        else: print("Weird file type:", vars(item))
    
    if __name__ == '__main__':
        starttime = time.time()
        pool = multiprocessing.Pool(2)
        pool.map(downloader, files)
        pool.close()
        print(f'{len(files)} items downloaded in {path} in {(time.time() - starttime)} seconds.')

#        for item in files:
#            print(item.name, 'process starting....')
#            p = multiprocessing.Process(target=downloader, args=(path, item))
#            processes.append(p)
#            p.start()

        #print(processes)

#        for process in processes:
#            process.join()

#        print(f'{len(processes)} items downloaded in {path} in {(time.time() - starttime)} seconds.')
        
    for item in sub_dirs:
        copy_box(path/str(item.name), item.id)        
        
        #####
    

In [32]:
copy_box('/Volumes/2TB_nvme/init_box_download_multi')

0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi in 0.008804082870483398 seconds.
0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi/Air_Pollution in 0.009589910507202148 seconds.
0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi/Air_Pollution/AFS in 0.013540029525756836 seconds.
0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi/Air_Pollution/Air Data in 0.010238885879516602 seconds.
0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi/Air_Pollution/Air Data/PM25_1990_1999 in 0.013396978378295898 seconds.
0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi/Air_Pollution/Air Data/PowerPlants_CO2 in 0.009846210479736328 seconds.
0 items downloaded in /Volumes/2TB_nvme/init_box_download_multi/Air_Pollution/AirModelComparison_JimAir in 0.009410858154296875 seconds.


TypeError: can't pickle _thread.lock objects

In [30]:
folder_items = client.folder(folder_id='93670449030').get_items()

In [31]:
for item in folder_items:
    print('{0} {1} is named "{2}"'.format(item.type.capitalize(), item.id, item.name))

Folder 93675974650 is named "Air_Pollution"
Folder 111822616290 is named "Boundaries"
Folder 93676481690 is named "Chemicals_Hazardous_Waste"
Folder 93676982303 is named "Drinking_Water"
Folder 93101593845 is named "Fallout EPA"
Folder 93675385253 is named "Ground_Water"
Folder 93676991156 is named "Heavy_Metals"
Folder 107607504405 is named "ICPSR_RAND"
Folder 112756166570 is named "LandCover"
Folder 93845957418 is named "Miscellaneous"
Folder 93676743393 is named "Pesticides"
Folder 93674659485 is named "Radiation"
Folder 93676016444 is named "Radon"
Folder 93674675131 is named "Superfunds"
Folder 112888553460 is named "VULCAN_FFCO2"
Folder 112747043084 is named "WalkabilityIndex"
File 659798412085 is named "Metadata Fields.docx"
File 619412509120 is named "READ_ME.xlsx"


In [52]:
file_obj = client.file(file_id='619412509120').get()
print(vars(file_obj))
print(type(file_obj.id))
output_file = open('test_download2_'+str(file_obj.name), 'wb')
file_obj.download_to(output_file)
output_file.close()

{'_response_object': {'type': 'file', 'id': '619412509120', 'file_version': <Box FileVersion - 707731312867>, 'sequence_id': '73', 'etag': '73', 'sha1': 'dd2c7f07da2dc163409ff6a8fb61e4c71c40af58', 'name': 'READ_ME.xlsx', 'description': '', 'size': 55357, 'path_collection': {'total_count': 2, 'entries': [<Box Folder - 0 (All Files)>, <Box Folder - 93670449030 (Environmental_Data_MASTER)>]}, 'created_at': '2020-02-19T08:03:13-08:00', 'modified_at': '2020-05-19T15:37:47-07:00', 'trashed_at': None, 'purged_at': None, 'content_created_at': '2020-02-19T08:02:24-08:00', 'content_modified_at': '2020-05-19T15:36:10-07:00', 'created_by': <Box User - 230268809 (Heidi Anne Hanson)>, 'modified_by': <Box User - 262768733 (JOEMY MARIE RAMSAY)>, 'owned_by': <Box User - 7664673418 (JACOB AMBROSE)>, 'shared_link': None, 'parent': <Box Folder - 93670449030 (Environmental_Data_MASTER)>, 'item_status': 'active'}, 'type': 'file', 'id': '619412509120', 'file_version': <Box FileVersion - 707731312867>, 'seque

In [20]:
for item in folder_items:
    print(type(item))

<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.folder.Folder'>
<class 'boxsdk.object.file.File'>
<class 'boxsdk.object.file.File'>


In [4]:
json_response = client.make_request(
    'GET',
    client.get_url('metadata_templates', 'enterprise', 'customer', 'schema'),
).json()


"GET https://api.box.com/2.0/metadata_templates/enterprise/customer/schema" 404 157
{'Date': 'Mon, 15 Jun 2020 04:16:03 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000', 'Cache-Control': 'no-cache, no-store', 'Content-Encoding': 'gzip', 'BOX-REQUEST-ID': '09c5395ab92f47e1ed4d94fb6bb9770e5'}
{'code': '---ound',
 'help_url': 'http://developers.box.com/docs/#errors',
 'message': 'Not Found',
 'request_id': 'qlczxgfsupn6a4m',
 'status': 404,
 'type': 'error'}



BoxAPIException: Message: Not Found
Status: 404
Code: not_found
Request ID: qlczxgfsupn6a4m
Headers: {'Date': 'Mon, 15 Jun 2020 04:16:03 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000', 'Cache-Control': 'no-cache, no-store', 'Content-Encoding': 'gzip', 'BOX-REQUEST-ID': '09c5395ab92f47e1ed4d94fb6bb9770e5'}
URL: https://api.box.com/2.0/metadata_templates/enterprise/customer/schema
Method: GET
Context Info: None

In [3]:
from boxsdk import OAuth2
